**Homework 26**

In this assignment you'll explore a sentiment analysis task. We start with the usual imports:


In [1]:
import torch
from torch import nn
from torch.optim import Adam
import numpy as np

device=('cuda' if torch.cuda.is_available()
        else 'mps' if torch.backends.mps.is_available()
        else 'cpu')

device

'cpu'

We now load the imdb movie review dataset that is packaged with keras. We'll only load the first 100 words of each review (if they are longer), and restrict to a 10,000 word vocabulary. The target variable y will be an array of 0's and 1's, indicating a positive or negative review.  

In [2]:
!pip install keras numpy==1.24.3
!pip uninstall tensorflow -y
!pip install tensorflow==2.14.0
# import keras
# from keras.datasets import imdb
from tensorflow import keras
from tensorflow.keras.datasets import imdb

(Xtrain,ytrain),(Xtest,ytest)=imdb.load_data(maxlen=100,num_words=10000,index_from=0)


[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: C:\Users\Jeremy Tan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached tensorflow-2.14.0-cp310-cp310-win_amd64.whl (2.1 kB)
     ---------------------------------------- 0.0/284.1 MB ? eta -:--:--
     --------------------------------------- 0.3/284.1 MB 10.2 MB/s eta 0:00:28
     --------------------------------------- 0.9/284.1 MB 14.5 MB/s eta 0:00:20
     --------------------------------------- 1.4/284.1 MB 12.8 MB/s eta 0:00:23
     --------------------------------------- 2.0/284.1 MB 12.5 MB/s eta 0:00:23
     --------------------------------------- 2.7/284.1 MB 12.1 MB/s eta 0:00:24
     --------------------------------------- 3.5/284.1 MB 13.8 MB/s eta 0:00:21
      -------------------------------------- 3.9/284.1 MB 12.4 MB/s eta 0:00:23
      -------------------------------------- 4.4/284.1 MB 12.9 MB/s eta 0:00:22
      -------------------------------------- 5.3/284.1 MB 13.5 MB/s eta 0:00:21
      -------------------------------------- 6.0/284.1 MB 13.7 MB/s eta 0:00:21
      -------------------------------------- 7.2/284.1 MB 1


[notice] A new release of pip is available: 23.0.1 -> 25.1
[notice] To update, run: C:\Users\Jeremy Tan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


17464789/17464789 [==============================] - 1s 0us/step


This dataset also comes with a dictionary for tokenization. The keys for this dictionary are the words in the vocabulary, and the values are the numbers assigned to each word (just like the `tokens` dictionary attribute of your `Tokenizer` class from the last assignment). We import this dictionary as `index`, and look at the first 10 key/value pairs:

In [3]:
index=imdb.get_word_index()
list(index.items())[:10]

1641221/1641221 [==============================] - 0s 0us/step


[('fawn', 34701),
 ('tsukino', 52006),
 ('nunnery', 52007),
 ('sonja', 16816),
 ('vani', 63951),
 ('woods', 1408),
 ('spiders', 16115),
 ('hanging', 2345),
 ('woody', 2289),
 ('trawling', 52008)]

Note that Xtrain[0] is not the words of the first movie review of the dataset, but rather the indices of those words. To see the review itself, we have to convert from indices back to words. To this end, we'll build a dictionary word_from_index whose keys are indices and values are corresponding words.

In [4]:
words_from_index={}
for word in index:
  words_from_index[index[word]]=word

We can now look at review 1:

In [5]:
review1=''
for n in Xtrain[1]:
  review1+=words_from_index[n]+' '

review1

"the when i rented this movie i had very low expectations but when i saw it i realized that the movie was less a lot less than what i expected the actors were bad the doctor's wife was one of the worst the story was so stupid it could work for a disney movie except for the murders but this one is not a comedy it is a laughable masterpiece of stupidity the title is well chosen except for one thing they could add stupid movie after dead husbands i give it 0 and a half out of 5 "

Right now Xtrain is a 1-dimensional numpy array of python lists. To convert it to a 2-dimensional numpy array, we'll need every every element of Xtrain to be a list of the same length. Write a function that takes a list-of-lists, and adds an appropriate number of 0's to each if their length is less than 100. (This is called *padding*.)

In [8]:
def pad(X):
  '''X is a list of lists of all different lengths
  function should return a list of lists of length 100 by padding with 0's'''
  return [x + [0] * (100 - len(x)) if len(x) < 100 else x[:100] for x in X]


We'll now use this function to convert Xtrain and Xtest to 2D numpy arrays. (At the same time we convert ytrain and ytest to numpy arrays as well.)

In [9]:
Xtrain=np.array([np.array(l) for l in pad(Xtrain)], dtype=np.int64)
Xtest=np.array([np.array(l) for l in pad(Xtest)], dtype=np.int64)

Finally, we convert all datasets to tensors and move them to the appropriate `device`.

In [10]:
X_train = torch.from_numpy(Xtrain).to(device)
y_train = torch.from_numpy(ytrain).to(device)
X_test  = torch.from_numpy(Xtest).to(device)
y_test  = torch.from_numpy(ytest).to(device)

In this homework we introduce the Embedding layer, which has the same effect as a one-hot encoding followed by a dense layer. We also use a GRU layer in place of a RNN (feel free to try an RNN or an LSTM instead).

Your model will consist of a Sequential Neural network with an Embedding layer and a GRU layer. For the embedding layer, the input_dim should be the size of your vocabulary, and the output_dim should be about 128 (you can play with that number). use 64 for the hidden dimension of the GRU, and don't forget to set `batch_first=True`.

Follow this sequential model with a separate, fully-connected linear layer with an appropriate output size for this task.

In [11]:
model = nn.Sequential(
    nn.Embedding(num_embeddings=10000  , embedding_dim=128),
    nn.GRU(input_size=128, hidden_size=64, batch_first=True)
).to(device)

fc = nn.Linear(64, 1).to(device)


Define an optimizer that will simultaneously adjust the parameters of both `model` and `fc`.

In [12]:
opt=Adam(list(model.parameters()) + list(fc.parameters()))


Define a loss function:

In [13]:
criterion = nn.BCEWithLogitsLoss()

Write your training loop! Use 100 epochs, with batches of size 32. Report losses as you train.

In [14]:
epochs = 100
batch_size = 32
num_batches = len(X_train) // batch_size

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i in range(0, len(X_train), batch_size):
        xb = X_train[i:i+batch_size]
        yb = y_train[i:i+batch_size].float().unsqueeze(1)

        opt.zero_grad()
        out, _ = model(xb)
        final_hidden_state = out[:, -1, :]  # get last GRU output
        logits = fc(final_hidden_state)

        loss = criterion(logits, yb)
        loss.backward()
        opt.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/num_batches:.4f}")

Epoch 1, Loss: 0.6981
Epoch 2, Loss: 0.6877
Epoch 3, Loss: 0.6227
Epoch 4, Loss: 0.4404
Epoch 5, Loss: 0.2755
Epoch 6, Loss: 0.1876
Epoch 7, Loss: 0.1325
Epoch 8, Loss: 0.0956
Epoch 9, Loss: 0.0649
Epoch 10, Loss: 0.0478
Epoch 11, Loss: 0.0411
Epoch 12, Loss: 0.0303
Epoch 13, Loss: 0.0342
Epoch 14, Loss: 0.0340
Epoch 15, Loss: 0.0275
Epoch 16, Loss: 0.0182
Epoch 17, Loss: 0.0160
Epoch 18, Loss: 0.0090
Epoch 19, Loss: 0.0026
Epoch 20, Loss: 0.0019
Epoch 21, Loss: 0.0009
Epoch 22, Loss: 0.0006
Epoch 23, Loss: 0.0005
Epoch 24, Loss: 0.0004
Epoch 25, Loss: 0.0004
Epoch 26, Loss: 0.0003
Epoch 27, Loss: 0.0003
Epoch 28, Loss: 0.0003
Epoch 29, Loss: 0.0002
Epoch 30, Loss: 0.0002
Epoch 31, Loss: 0.0002
Epoch 32, Loss: 0.0002
Epoch 33, Loss: 0.0002
Epoch 34, Loss: 0.0002
Epoch 35, Loss: 0.0001
Epoch 36, Loss: 0.0001
Epoch 37, Loss: 0.0001
Epoch 38, Loss: 0.0001
Epoch 39, Loss: 0.0001
Epoch 40, Loss: 0.0001
Epoch 41, Loss: 0.0001
Epoch 42, Loss: 0.0001
Epoch 43, Loss: 0.0001
Epoch 44, Loss: 0.00

Evaluate the accuracy of your model on Xtest and ytest. (It should be around 80%.)

In [15]:
# model.eval()
# accuracy=#YOUR CODE HERE
model.eval()
with torch.no_grad():
    out, _ = model(X_test)
    final_hidden = out[:, -1, :]
    logits = fc(final_hidden)
    preds = (torch.sigmoid(logits) > 0.5).squeeze()
    accuracy = (preds == y_test).float().mean().item()
accuracy

0.8015524744987488

Take a screen shot of this accuracy and upload to Gradescope!